In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import joblib
import sasc.config
import numpy as np
from collections import defaultdict
import pandas as pd
from sasc.modules.fmri_module import convert_module_num_to_voxel_num, add_stability_score
from sasc.config import FMRI_DIR, STORIES_DIR

# Read all the info from stories into a single pickle file

In [2]:
# load stuff
# double check all of these, intro paragraph may be the same...
output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot5_story_data.pkl")
story_mapping = {
    'roi/uts02___roi_may31___seed=5_best1': 'deeptune-story23.npy',
    'roi/uts02___roi_may31___seed=2_best2': 'deeptune-story24.npy',
    'roi/uts02___roi_may31___seed=7_best3': 'deeptune-story25.npy',
    'roi/uts02___roi_may31___seed=6_best4': 'deeptune-story26.npy',

    # todo
    # 'qa/uts02___qa_may31___seed=1': 'GenStory27_resps.npy',
    # 'qa/uts02___qa_may31___seed=2': 'GenStory28_resps.npy',
    # 'qa/uts02___qa_may31___seed=3': 'GenStory29_resps.npy',
}

In [3]:
# cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))
perfs = joblib.load(join(sasc.config.FMRI_DIR, 'sasc', 'rj_models',
                    'opt_model', 'new_setup_performance.jbl'))

# add keys
stories_data_dict = defaultdict(list)
for story_idx, story_name in enumerate(story_mapping.keys()):
    # add scalar story descriptions
    stories_data_dict["story_name_original"].append(story_name)
    stories_data_dict["story_setting"].append(story_name.split("/")[0])
    stories_data_dict["story_name_new"].append(story_mapping[story_name])
    stories_data_dict["story_text"].append(
        open(join(STORIES_DIR, story_name, "story.txt"), "r").read()
    )
    prompts_paragraphs = joblib.load(
        join(STORIES_DIR, story_name, "prompts_paragraphs.pkl")
    )

    # add paragraph-level descriptions
    stories_data_dict["timing"].append(
        pd.read_csv(join(STORIES_DIR, story_name, "timings_processed.csv"))
    )
    stories_data_dict["prompts"].append(prompts_paragraphs["prompts"])
    stories_data_dict["paragraphs"].append(prompts_paragraphs["paragraphs"])

    # add paragraph-level metadata
    # rows
    # rows = pd.read_csv(join(STORIES_DIR, story_name, "rows.csv"))
    story_metadata_per_paragraph = pd.read_pickle(
        join(STORIES_DIR, story_name, "rows.pkl"))
    # story_metadata_per_paragraph["voxel_num"] = story_metadata_per_paragraph.apply(
    # lambda row: convert_module_num_to_voxel_num(
    # row["module_num"], row["subject"]),
    # axis=1,
    # )
    # story_metadata_per_paragraph["stability_score"] = story_metadata_per_paragraph.apply(
    # lambda row: add_stability_score(row["module_num"], row["subject"]),
    # axis=1,
    # )
    # story_metadata_per_paragraph = story_metadata_per_paragraph[
    #     [
    #         "expl",
    #         "module_num",
    #         "top_explanation_init_strs",
    #         "subject",
    #         "fmri_test_corr",
    #         "stability_score",
    #         # "top_score_synthetic",
    #         "top_score_normalized",
    #         "roi_anat",
    #         "roi_func",
    #         "voxel_num",
    #     ]
    # ]
    # story_metadata_per_paragraph['test_corr_new'] = story_metadata_per_paragraph['voxel_num'].apply(
    #     lambda x: perfs[x])
    # rows['cluster_nums'] = rows['voxel_num'].map(cluster_neighbors)
    stories_data_dict["rows"].append(story_metadata_per_paragraph)

joblib.dump(stories_data_dict, output_file)

['/home/chansingh/automated-explanations/results/processed/pilot5_story_data.pkl']

### extra exploration

In [ ]:
metadata = pd.read_pickle('../0_voxel_select/rows_roi_uts02_may31.pkl')
metadata

In [ ]:
metadata['expl'].unique().shape

In [ ]:
story_metadata_per_paragraph['expl'].unique().shape

In [ ]:
# merge
assert story_metadata_per_paragraph['expl'].unique(
).shape == metadata['expl'].unique().shape
story_metadata_per_paragraph = pd.merge(
    story_metadata_per_paragraph, metadata[['expl', 'roi', 'voxel_nums', 'stability_score']], on=['expl'], how='inner', )

In [ ]:
story_metadata_per_paragraph